In [1]:
import ipywidgets as w
from traitlets import HasTraits, List, Tuple, Bool,Int,Instance,observe

In [44]:
class Hour(HasTraits):
    value = Bool()

class Day:
    def __init__(self):
        self.hours = [Hour() for _ in range(24)]

class Calendar:

    def __init__(self,ndays):
        self.days = [Day() for _ in range(ndays)]

class CalendarView:

    def __init__(self,model):
        self.model = model
        self.days = [ DayView(d) for d in self.model.days]
        self.view = w.HBox([d.view for d in self.days])

    def _ipython_display_(self):
        display(self.view)

class DayView:

    l = w.Layout(width="90%",height="100%")
    
    def __init__(self,model):
        self.model = model
        self.hours = [w.ToggleButton(description=("" if (i+1) % 4 else str(i+1)),layout=l,disabled=True) for i,_ in enumerate(model.hours)]

        for v,m in zip(self.hours,self.model.hours):
            w.link((v,"value"),(m,"value"))

        self.view = w.VBox(self.hours)

    def _ipython_display_(self):
        display(self.view)

In [45]:
class AppModel(HasTraits):
    
    ndays = Int()
    hours = Int(1)
    start = Int()
    calendar = Instance(Calendar,default_value=Calendar(7))
    _calc = Bool()

    def __init__(self,ndays):
        self.start = 1
        self.ndays = ndays
        self._calc = False
        # self.create_calendar()
    
    @observe("ndays")
    def on_ndays(self,change):
        # print(change)
        if not self._calc and self.calendar:
            self._calc = True
            self.create_calendar()
            self._calc = False

    @observe("start")
    def on_start(self,change):
        # print(change)
        if not self._calc and self.calendar:
            self._calc = True
            self.create_calendar()
            self._calc = False

    @observe("hours")
    def on_hours(self,change):
        # print(change)
        if not self._calc and self.calendar:
            self._calc = True
            self.create_calendar()
            self._calc = False

    def create_calendar(self):
        self.calendar = Calendar(self.ndays)
        hours = (h for d in self.calendar.days for h in d.hours )
        st = self.g(self.ndays, self.start,self.hours)
        for h,v in zip(hours,st):
            h.value = v
        # self.calendar.days[0].hours[self.start-1].value = True

    def g(self,ndays,start,hours):
        return ( ((h-start) % hours) == hours-1 if h>=start-1 else False for h in range(24*ndays))

In [46]:
class App:

    def __init__(self,ndays=7):
        # self.initial_ndays=ndays
        self.model= AppModel(ndays)
        self.ndays = w.IntSlider(value=ndays,max=7, description="ndays",disabled=False)
        self.hours = w.IntSlider(max=48, description="hours")
        self.start = w.IntSlider(min=1,max=24, description="start")

        w.link((self.model,"ndays"),(self.ndays,"value"))
        w.link((self.model,"hours"),(self.hours,"value"))
        w.link((self.model,"start"),(self.start,"value"))
         
        self.view = w.Box([w.VBox([w.HBox([self.ndays,self.hours,self.start]), CalendarView(self.model.calendar).view])])

        self.model.observe(self.update_calendar,names="calendar")
        
    def update_calendar(self, change):
        self.view.children =  [w.VBox([w.HBox([self.ndays,self.hours,self.start]), CalendarView(self.model.calendar).view])]
        # self.view = w.VBox([w.HBox([self.ndays,self.hours,self.start]), self.calendar_view.view])


    def _ipython_display_(self):
        display(self.view)

In [47]:
a=App(2)

In [48]:
a

Box(children=(VBox(children=(HBox(children=(IntSlider(value=2, description='ndays', max=7), IntSlider(value=1,…